In [1]:
import os
import time
import warnings
import math
import argparse 
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler 

warnings.filterwarnings('ignore')

import sys
project_root = os.path.abspath('./') 
if project_root not in sys.path:
    sys.path.append(project_root)

from exp.exp_main import Exp_Main
from models import xPatch 
from data_provider.data_factory import data_provider
from data_provider.data_loader import Dataset_Custom 
from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric

In [6]:
class Args:
    def __init__(self):
        self.is_training = 1
        self.train_only = False
        self.model_id = 'AAPL_Pred5_Notebook_Tuned'
        self.model = 'xPatch'

        self.data = 'custom'
        self.scale = True
        self.root_path = './data/' 
        #self.data_path = 'aapl_OHLCV.csv'
        self.data_path = 'ETTh1.csv'
        self.features = 'MS' # Multiple Features, Single target
        #self.target = 'Close' # The target feature
        self.target = 'OT' # The target feature
        self.freq = 'h'
        self.checkpoints = './checkpoints/' 
        self.embed = 'timeF' 

####################################################################################################################################################
# Hyperparameter Tuning
        # Forecasting Task
        self.seq_len = 48
        self.label_len = 48
        self.pred_len = 5
        self.enc_in = 7 # Ensure this matches the number of features in the CSV after 'date'

        # Patching
        self.patch_len = 12
        self.stride = 6
        self.padding_patch = 'end'

        # Moving Average
        self.ma_type = 'ema'
        self.alpha = 0.2
        self.beta = 0.2

        # Optimization
        self.num_workers = 8 # Adjust based on your system 
        self.itr = 1 
        self.train_epochs = 3
        self.batch_size = 16
        self.patience = 15
        self.learning_rate = 0.0001
        self.des = 'Notebook_Run_MS'
        self.loss = 'mae' 
        self.lradj = 'type1' # learning rate strategy
        self.revin = 1 
####################################################################################################################################################

        # GPU
        self.use_gpu = True if torch.cuda.is_available() else False
        self.gpu = 0 
        self.use_multi_gpu = False 
        self.devices = '0' 
        self.test_flop = False 

args = Args()

if not os.path.exists(args.checkpoints):
    os.makedirs(args.checkpoints)

In [7]:

exp = Exp_Main(args)

print(f'Args for this run: {vars(args)}')

best_model = exp.train(args.model_id)


Use CPU
Args for this run: {'is_training': 1, 'train_only': False, 'model_id': 'AAPL_Pred5_Notebook_Tuned', 'model': 'xPatch', 'data': 'custom', 'scale': True, 'root_path': './data/', 'data_path': 'ETTh1.csv', 'features': 'MS', 'target': 'OT', 'freq': 'h', 'checkpoints': './checkpoints/', 'embed': 'timeF', 'seq_len': 48, 'label_len': 48, 'pred_len': 5, 'enc_in': 7, 'patch_len': 12, 'stride': 6, 'padding_patch': 'end', 'ma_type': 'ema', 'alpha': 0.2, 'beta': 0.2, 'num_workers': 8, 'itr': 1, 'train_epochs': 3, 'batch_size': 16, 'patience': 15, 'learning_rate': 0.0001, 'des': 'Notebook_Run_MS', 'loss': 'mae', 'lradj': 'type1', 'revin': 1, 'use_gpu': False, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0', 'test_flop': False}
train 12142
val 1738
test 3480
	iters: 100, epoch: 1 | loss: 0.1634597
	speed: 0.0087s/iter; left time: 18.8641s
	iters: 200, epoch: 1 | loss: 0.1719241
	speed: 0.0078s/iter; left time: 16.2717s
	iters: 300, epoch: 1 | loss: 0.1640683
	speed: 0.0073s/iter; left time: 

In [4]:
print(args.model_id)
exp.test(args.model_id)


AAPL_Pred5_Notebook_Tuned
test 3480


[[[-1.4934598]
  [-1.486687 ]
  [-1.5125166]
  [-1.5161653]
  [-1.5153393]]

 [[-1.4901649]
  [-1.4835786]
  [-1.508399 ]
  [-1.5272821]
  [-1.5119267]]

 [[-1.5243843]
  [-1.5223712]
  [-1.5301377]
  [-1.5707573]
  [-1.5531691]]

 [[-1.4986303]
  [-1.4936461]
  [-1.5092046]
  [-1.5460184]
  [-1.5194241]]

 [[-1.5238769]
  [-1.5203973]
  [-1.5315751]
  [-1.550312 ]
  [-1.560586 ]]

 [[-1.5734307]
  [-1.5699741]
  [-1.5714597]
  [-1.6152024]
  [-1.6129472]]

 [[-1.5316964]
  [-1.5244361]
  [-1.540883 ]
  [-1.5659735]
  [-1.556776 ]]

 [[-1.4246144]
  [-1.4275099]
  [-1.4385841]
  [-1.4605101]
  [-1.4689997]]

 [[-1.4696546]
  [-1.4651889]
  [-1.4851145]
  [-1.4986157]
  [-1.5022726]]

 [[-1.482423 ]
  [-1.4731373]
  [-1.5029044]
  [-1.5035744]
  [-1.5060275]]

 [[-1.4397846]
  [-1.4357171]
  [-1.4586316]
  [-1.4761045]
  [-1.4584296]]

 [[-1.395969 ]
  [-1.3987411]
  [-1.4087958]
  [-1.4294549]
  [-1.4110425]]

 [[-1.4287487]
  [-1.4280237]
  [-1.4427853]
  [-1.4403977]
  [-1.4558446]]


In [1]:
folder_path = os.path.join('./test_results/', args.model_id + '/')
predictions_path = os.path.join(folder_path, 'pred.npy')
trues_path = os.path.join(folder_path, 'true.npy')

if os.path.exists(predictions_path) and os.path.exists(trues_path):
    all_preds = np.load(predictions_path)
    all_trues = np.load(trues_path)

    import matplotlib.pyplot as plt


    sample_idx = 10 # Pick a sample to visualize

    plt.figure(figsize=(12, 6))
    plt.plot(all_trues[sample_idx, :, 0], label='Ground Truth')
    plt.plot(all_preds[sample_idx, :, 0], label='Prediction')
    plt.title(f'Sample {sample_idx} Prediction vs Ground Truth (Last Channel)')
    plt.xlabel('Time Step (future days)')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(False)
    plt.show()

else:
    print("Prediction and true value files not found. Ensure test() ran successfully.")

NameError: name 'os' is not defined